In [1]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split


In [2]:
data_path = 'data/Training'
categories = os.listdir(data_path)
labels = [i for i in range(len(categories))]
label_dict = dict(zip(categories, labels))

print(label_dict)
print(categories)
print(labels)

{'pituitary': 0, 'notumor': 1, 'glioma': 2, 'meningioma': 3}
['pituitary', 'notumor', 'glioma', 'meningioma']
[0, 1, 2, 3]


In [3]:
img_size = 500
data = []
target = []

for category in categories:
    folder_path = os.path.join(data_path, category)
    img_names = os.listdir(folder_path)

    for img_name in img_names:
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)

        try:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray, (img_size, img_size))
            data.append(resized)
            target.append(label_dict[category])

        except Exception as e:
            print('Exception:', e)

In [4]:
import warnings
warnings.filterwarnings('ignore')
data = np.array(data) / 255.0
data = np.reshape(data, (data.shape[0], img_size, img_size, 1))
target = np.array(target)

from keras.utils import np_utils
new_target = np_utils.to_categorical(target)

np.save('training/data', data)
np.save('training/target', new_target)

2024-10-01 16:39:55.888544: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
data = np.load('training/data.npy')
target = np.load('training/target.npy')

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, earlyStopping

noOfFilters = 64
sizeOfFilter1 = (3, 3)
sizeOfFilter2 = (3, 3)
sizeOfPool = (2, 2)
noOfNodes = 128

model = Sequential()
model.add((Conv2D(noOfFilters, sizeOfFilter1, input_shape=(img_size, img_size, 1), activation='relu')))
model.add((Conv2D(noOfFilters, sizeOfFilter1, activation='relu')))
model.add(MaxPooling2D(pool_size=sizeOfPool))

model.add((Conv2D(noOfFilters, sizeOfFilter2, activation='relu')))
model.add((Conv2D(noOfFilters, sizeOfFilter2, activation='relu')))
model.add(MaxPooling2D(pool_size=sizeOfPool))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(noOfNodes, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','precision','recall'])

check_point = ModelCheckpoint('model.h5}', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
early_stop = earlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

In [7]:
train_data, test_data, train_target, test_target = train_test_split(data, target, test_size=0.1)

In [ ]:
history = model.fit(train_data, train_target, epochs=20, validation_split=0.2, verbose=1, callbacks=[check_point])
model.save('brainTumor.keras')